In [ ]:
from google.colab import files
import librosa
import torch
import numpy as np

# 음성 파일 업로드
uploaded = files.upload()

# 업로드된 파일 경로
audio_path = '음성 250421_122546.m4a'

# 음성을 로드하는 함수 (duration을 받을 수 있도록 수정)
def load_audio(file_path, sr=16000, duration=None):
    audio, _ = librosa.load(file_path, sr=sr, duration=duration)  # duration을 추가
    return audio

# Mel-spectrogram 전환
def mel_spectrogram(audio_data, sr=16000, n_mels=80):
    # Mel-spectrogram 변환
    mel_spec = librosa.feature.melspectrogram(y=audio_data, sr=sr, n_mels=n_mels)

    # dB 단위로 변환 (log scale)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    # Mel-spectrogram을 tensor로 변환
    mel_spec_tensor = torch.tensor(mel_spec_db).unsqueeze(0)

    return mel_spec_tensor

# 오디오 파일 로드
audio_data = load_audio(audio_path)

# Mel-spectrogram 변환
mel_spec = mel_spectrogram(audio_data)

Saving 음성 250421_122546.m4a to 음성 250421_122546 (3).m4a


<ipython-input-67-ef374b5dab2a>:14: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file_path, sr=sr, duration=duration)  # duration을 추가
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [ ]:
import torch.nn.functional as F

# Swish 함수 직접 정의
def swish(x):
    return x * torch.sigmoid(x)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# -----------------------------
# 1. FeedForward 모듈
# -----------------------------
class FeedForwardModule(nn.Module):
    def __init__(self, dim, expansion_factor=4, dropout=0.1):
        super().__init__()
        # FeedForward 네트워크 구성 (LayerNorm → Linear 확장 → ReLU → Dropout → Linear 축소 → Dropout)
        self.ffn = nn.Sequential(
            nn.LayerNorm(dim),                             # 입력 정규화 (Layer Normalization)
            nn.Linear(dim, dim * expansion_factor),        # 차원 확장 (예: 144 → 576)
            nn.ReLU(),                                     # 비선형 활성화 함수
            nn.Dropout(dropout),                           # 과적합 방지를 위한 드롭아웃
            nn.Linear(dim * expansion_factor, dim),        # 다시 원래 차원으로 축소
            nn.Dropout(dropout),                           # 드롭아웃 한 번 더 적용
        )

    def forward(self, x):
        # FeedForward 네트워크에 입력 전달
        return self.ffn(x)


# -----------------------------
# 2. Multi-Head Self-Attention 모듈
# -----------------------------
class MultiHeadSelfAttentionModule(nn.Module):
    def __init__(self, dim, num_heads, dropout=0.1):
        super().__init__()
        self.layer_norm = nn.LayerNorm(dim)  # 입력 정규화 (Layer Normalization)

        # PyTorch 내장 MultiheadAttention 모듈
        # embed_dim: 임베딩 차원, num_heads: attention head 수, batch_first=True로 입력 형태 (B, T, D) 처리
        self.attention = nn.MultiheadAttention(
            embed_dim=dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )

    def forward(self, x):
        x_norm = self.layer_norm(x)  # Layer Normalization 적용

        # Self-Attention 수행
        # query, key, value 모두 동일한 입력 (Self-Attention이므로)
        attn_output, _ = self.attention(x_norm, x_norm, x_norm)

        # Attention 결과 반환 (Residual 연결은 외부에서 수행)
        return attn_output

# -----------------------------
# 3. Convolution 모듈
# -----------------------------
class ConvolutionModule(nn.Module):
    def __init__(self, dim, kernel_size=31, dropout=0.1):
        super().__init__()
        self.layer_norm = nn.LayerNorm(dim)  # 입력 정규화

        # Pointwise Convolution (1x1 Conv)로 채널 수 2배로 증가
        self.pointwise_conv1 = nn.Conv1d(dim, 2 * dim, kernel_size=1)

        # Gated Linear Unit (GLU): 게이트를 통해 정보 흐름 제어
        self.glu = nn.GLU(dim=1)

        # Depthwise Convolution: 채널별로 독립적인 커널 적용 (병렬성 및 파라미터 효율성↑)
        self.depthwise_conv = nn.Conv1d(
            dim, dim, kernel_size=kernel_size,
            padding=kernel_size // 2,  # 출력 길이 유지
            groups=dim  # 채널별 depthwise
        )

        self.batch_norm = nn.BatchNorm1d(dim)  # 배치 정규화
        self.pointwise_conv2 = nn.Conv1d(dim, dim, kernel_size=1)  # 채널 수 복원
        self.dropout = nn.Dropout(dropout)  # 드롭아웃

    def forward(self, x):
        x = self.layer_norm(x)     # 정규화
        x = x.transpose(1, 2)      # (B, T, D) -> (B, D, T), Conv1D 입력 형식에 맞춤
        x = self.pointwise_conv1(x)  # 1x1 conv로 채널 2배
        x = self.glu(x)            # 게이트로 정보 제어
        x = self.depthwise_conv(x)  # Depthwise conv 적용
        x = self.batch_norm(x)     # 배치 정규화
        x = swish(x)               # 비선형 활성화 함수 (Swish)
        x = self.pointwise_conv2(x)  # 채널 수 원래대로
        x = self.dropout(x)        # 드롭아웃 적용
        x = x.transpose(1, 2)      # (B, D, T) -> (B, T, D) 복원
        return x

# -----------------------------
# 4. Conformer Block
# -----------------------------
class ConformerBlock(nn.Module):
    def __init__(self, dim, num_heads, ff_expansion_factor=4, conv_kernel_size=31, dropout=0.1):
        super().__init__()

        # 첫 번째 FeedForward 모듈 (Pre-Attention)
        self.ffn1 = FeedForwardModule(dim, ff_expansion_factor, dropout)

        # Multi-Head Self-Attention 모듈
        self.self_attn = MultiHeadSelfAttentionModule(dim, num_heads, dropout)

        # Convolution 모듈 (로컬 정보 학습)
        self.conv = ConvolutionModule(dim, conv_kernel_size, dropout)

        # 두 번째 FeedForward 모듈 (Post-Convolution)
        self.ffn2 = FeedForwardModule(dim, ff_expansion_factor, dropout)

        # 최종 Layer Normalization
        self.final_norm = nn.LayerNorm(dim)

    def forward(self, x):
        # 첫 번째 FeedForward는 스케일을 0.5로 줄여 더해줌 (Macaron 스타일)
        x = x + 0.5 * self.ffn1(x)

        # Self-Attention 모듈을 거쳐 장기 의존성 학습
        x = x + self.self_attn(x)

        # Convolution 모듈로 로컬 정보 학습
        x = x + self.conv(x)

        # 두 번째 FeedForward도 0.5 스케일로 더함
        x = x + 0.5 * self.ffn2(x)

        # 최종적으로 LayerNorm 적용
        x = self.final_norm(x)
        return x

# -----------------------------
# 5. Encoder (여러 Conformer Block 쌓기)
# -----------------------------
class ConformerEncoder(nn.Module):
    def __init__(self, input_dim, model_dim, num_layers, num_heads):
        super().__init__()

        # 입력 차원(input_dim)을 모델 차원(model_dim)으로 변환
        # 예: mel-spectrogram feature (예: 80차원) → 모델 내 처리용 차원 (예: 144차원)
        self.input_linear = nn.Linear(input_dim, model_dim)

        # 여러 개의 Conformer 블록을 쌓은 구조
        self.blocks = nn.ModuleList([
            ConformerBlock(model_dim, num_heads) for _ in range(num_layers)
        ])

    def forward(self, x):
        # 입력 feature 차원 변환
        x = self.input_linear(x)

        # 순차적으로 모든 Conformer 블록을 통과
        for block in self.blocks:
            x = block(x)

        # 인코딩된 최종 특징 반환
        return x

# -----------------------------
# 6. 전체 STT 파이프라인 예시
# -----------------------------
class SimpleSTTModel(nn.Module):
    def __init__(self, input_dim, model_dim, num_layers, num_heads, num_classes):
        super().__init__()

        # Conformer 기반 Encoder 정의
        # 음성의 시간-주파수 정보를 인코딩해주는 역할
        self.encoder = ConformerEncoder(input_dim, model_dim, num_layers, num_heads)

        # 인코더 출력(feature map)을 최종 클래스 개수로 투영
        # 각 시간 스텝마다 문자(class)를 예측하게 됨
        self.output_layer = nn.Linear(model_dim, num_classes)

    def forward(self, x):
        # Conformer 인코더를 거쳐 특징 추출
        encoded = self.encoder(x)

        # 각 시점(time step)에 대해 class 개수만큼의 로짓(logit) 출력
        logits = self.output_layer(encoded)

        return logits  # (batch, time, num_classes)

# 예시 사용
if __name__ == "__main__":
    # 입력 예시 (batch=1, time=100, feature=80)
    # 예: mel-spectrogram (100 프레임, 각 프레임당 80차원 특징)
    # mel_spec: (1, 80, T) → (1, T, 80) 로 바꿔야 함
    mel_spec = mel_spec.transpose(1, 2)
    dummy_input = mel_spec

    # 모델 초기화: 80차원 입력, 모델 내부는 144차원, 블록 4개, attention head 4개, 출력 클래스 5000개
    model = SimpleSTTModel(input_dim=80, model_dim=144, num_layers=4, num_heads=4, num_classes=5000)

    # forward 수행 (예측)
    output = model(dummy_input)

    print("Output shape:", output.shape)  # 예: (1, 100, 5000)

Output shape: torch.Size([1, 485, 5000])


In [ ]:
# 한글 자모와 완성형 한글을 포함한 vocab 생성
def create_vocab():
    # 한글 음절
    hangul = [chr(i) for i in range(ord('가'), ord('힣')+1)]
    # 자음과 모음
    consonants = ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    vowels = ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ']
    # 공백과 [UNK] 추가
    vocab = ['<pad>', '<sos>', '<eos>', '<unk>'] + consonants + vowels + hangul
    return vocab

# vocab 생성
vocab = create_vocab()
print(len(vocab), vocab[:20])  # vocab의 길이와 처음 20개 문자 출력

11200 ['<pad>', '<sos>', '<eos>', '<unk>', 'ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㅏ', 'ㅑ']


In [ ]:
import torch

# CTC 디코딩 함수 (Greedy Decoding)
def greedy_decode(output, vocab):
    # 모델 출력에서 각 타임스텝에서 가장 높은 확률을 가진 인덱스를 선택
    _, max_indices = torch.max(output, dim=-1)  # 각 타임스텝에서 가장 높은 확률 선택

    # max_indices는 2D 텐서이므로, 이를 1D 텐서로 변경하여 각 인덱스를 문자로 변환
    max_indices = max_indices.squeeze(0)  # 배치 차원 제거 (배치 크기가 1일 경우)

    # 인덱스를 문자로 변환 (vocab 인덱스를 통해 문자 매핑)
    decoded_text = ''.join([vocab[i.item()] for i in max_indices])  # 인덱스를 문자로 변환

    # 반복된 문자와 <pad>, <sos>, <eos> 처리
    decoded_text = decoded_text.replace('<pad>', '').replace('<sos>', '').replace('<eos>', '')
    decoded_text = ''.join([decoded_text[i] if i == 0 or decoded_text[i] != decoded_text[i-1] else '' for i in range(len(decoded_text))])  # 중복 문자 제거

    return decoded_text



# 모델 초기화: 80차원 입력, 모델 내부는 144차원, 블록 4개, attention head 4개, 출력 클래스 5000개
model = SimpleSTTModel(input_dim=80, model_dim=144, num_layers=4, num_heads=4, num_classes=5000)

# 모델 예측
model.eval()
with torch.no_grad():
    output = model(mel_spec)            # (1, T, num_classes)
print("예측 결과:", decoded)

#decoded_text = greedy_decode(output, vocab)  # CTC 디코딩
#print("Predicted Text: ", decoded_text)  # 예측된 텍스트 출력

예측 결과: 껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤룶룶룶룶룶룶껤룶룶룶룶껤껤껤껤껤껤껤껤껤껤껤껤껤룶룶껤껤껤룶룶룶룶룶룶룶룶껤룶룶껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶껤껤껤껤껤껤껤룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶껤껤룶룶룶껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤룶룶룶룶껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤룶껤껤껤껤껤껤껤껤룶룶룶룶룶껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤룶룶룶룶룶룶룶룶룶껤껤룶룶껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶껤껤껤껤껤룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶룶껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤껤


In [ ]:
# 음성을 녹음하여 Mel-spectrogram으로 변환하고, 모델에 입력하여 텍스트를 예측하는 전체 파이프라인

def stt_pipeline():
    audio_file = '/content/음성 250421_122546.m4a'  # 음성 파일 경로 지정
    audio_data = load_audio(audio_file, duration=5)  # 파일 경로와 duration 인자 전달

    # 녹음된 음성을 Mel-spectrogram으로 변환
    mel_spec = mel_spectrogram(audio_data)

    # 이후 모델을 사용하여 텍스트 예측 및 디코딩 진행
    output = mel_spec
    decoded_text = greedy_decode(output, vocab)
    print("Predicted Text: ", decoded_text)

# 파이프라인 실행
stt_pipeline()

Predicted Text:  갅걸갈갋걲걟갊걹걽갗갖걠갢갚감걦개감갏갢갌갏갣갢갏개갍갎갏갌갏감갋감갋갊갋갊갖갊갗갋갏갊걮갔갋값갓걮갓갗값


<ipython-input-67-ef374b5dab2a>:14: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file_path, sr=sr, duration=duration)  # duration을 추가
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [ ]:
#한번에 정리

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

# ---------------------------------------------------
# 0. 유틸 함수
# ---------------------------------------------------
def swish(x):
    return x * torch.sigmoid(x)

# ---------------------------------------------------
# 1. FeedForward Module
# ---------------------------------------------------
class FeedForwardModule(nn.Module):
    def __init__(self, dim, expansion_factor=4, dropout=0.1):
        super().__init__()
        self.ffn = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, dim * expansion_factor),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(dim * expansion_factor, dim),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.ffn(x)

# ---------------------------------------------------
# 2. Multi-Head Self Attention Module
# ---------------------------------------------------
class MultiHeadSelfAttentionModule(nn.Module):
    def __init__(self, dim, num_heads, dropout=0.1):
        super().__init__()
        self.layer_norm = nn.LayerNorm(dim)
        self.attention = nn.MultiheadAttention(
            embed_dim=dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )

    def forward(self, x):
        x_norm = self.layer_norm(x)
        attn_output, _ = self.attention(x_norm, x_norm, x_norm)
        return attn_output

# ---------------------------------------------------
# 3. Convolution Module
# ---------------------------------------------------
class ConvolutionModule(nn.Module):
    def __init__(self, dim, kernel_size=31, dropout=0.1):
        super().__init__()
        self.layer_norm = nn.LayerNorm(dim)
        self.pointwise_conv1 = nn.Conv1d(dim, 2 * dim, kernel_size=1)
        self.glu = nn.GLU(dim=1)
        self.depthwise_conv = nn.Conv1d(dim, dim, kernel_size=kernel_size, padding=kernel_size // 2, groups=dim)
        self.batch_norm = nn.BatchNorm1d(dim)
        self.pointwise_conv2 = nn.Conv1d(dim, dim, kernel_size=1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = x.transpose(1, 2)
        x = self.pointwise_conv1(x)
        x = self.glu(x)
        x = self.depthwise_conv(x)
        x = self.batch_norm(x)
        x = swish(x)
        x = self.pointwise_conv2(x)
        x = self.dropout(x)
        x = x.transpose(1, 2)
        return x

# ---------------------------------------------------
# 4. Conformer Block
# ---------------------------------------------------
class ConformerBlock(nn.Module):
    def __init__(self, dim, num_heads, ff_expansion_factor=4, conv_kernel_size=31, dropout=0.1):
        super().__init__()
        self.ffn1 = FeedForwardModule(dim, ff_expansion_factor, dropout)
        self.self_attn = MultiHeadSelfAttentionModule(dim, num_heads, dropout)
        self.conv = ConvolutionModule(dim, conv_kernel_size, dropout)
        self.ffn2 = FeedForwardModule(dim, ff_expansion_factor, dropout)
        self.final_norm = nn.LayerNorm(dim)

    def forward(self, x):
        x = x + 0.5 * self.ffn1(x)
        x = x + self.self_attn(x)
        x = x + self.conv(x)
        x = x + 0.5 * self.ffn2(x)
        x = self.final_norm(x)
        return x

# ---------------------------------------------------
# 5. Conformer Encoder
# ---------------------------------------------------
class ConformerEncoder(nn.Module):
    def __init__(self, input_dim, model_dim, num_layers, num_heads):
        super().__init__()
        self.input_linear = nn.Linear(input_dim, model_dim)
        self.blocks = nn.ModuleList([
            ConformerBlock(model_dim, num_heads) for _ in range(num_layers)
        ])

    def forward(self, x):
        x = self.input_linear(x)
        for block in self.blocks:
            x = block(x)
        return x

# ---------------------------------------------------
# 6. 전체 STT 모델
# ---------------------------------------------------
class SimpleSTTModel(nn.Module):
    def __init__(self, input_dim, model_dim, num_layers, num_heads, num_classes):
        super().__init__()
        self.encoder = ConformerEncoder(input_dim, model_dim, num_layers, num_heads)
        self.output_layer = nn.Linear(model_dim, num_classes)

    def forward(self, x):
        encoded = self.encoder(x)
        logits = self.output_layer(encoded)
        return logits  # (batch, time, num_classes)

# ---------------------------------------------------
# 7. 문자 디코딩 함수
# ---------------------------------------------------
import pandas as pd


def load_vocab(csv_path):
    df = pd.read_csv(csv_path)
    # 컬럼 이름 공백 제거 및 소문자 변환
    df.columns = df.columns.str.strip().str.lower()

    if 'id' not in df.columns or 'char' not in df.columns:
        print("👉 현재 컬럼:", df.columns)
        raise ValueError("CSV 파일에 'id' 또는 'char' 컬럼이 없습니다.")

    df['id'] = df['id'].astype(int)
    df['char'] = df['char'].astype(str)
    id2char = dict(zip(df['id'], df['char']))
    return id2char

def decode_predictions(pred_ids, id2char):
    ignore_tokens = [0, 1, 2]  # <pad>, <sos>, <eos>
    chars = [id2char[i.item()] for i in pred_ids if i.item() not in ignore_tokens and i.item() in id2char]
    return ''.join(chars)

# ---------------------------------------------------
# 8. 테스트 실행
# ---------------------------------------------------
if __name__ == "__main__":
    # 1. mel-spectrogram 데이터 로딩
    audio_file = '/content/음성 250421_122546.m4a'  # 음성 파일 경로 지정
    audio_data = load_audio(audio_file, duration=5)  # 파일 경로와 duration 인자 전달

    # 2. mel-spectrogram 변환 및 shape 정리
    mel_spec = mel_spectrogram(audio_data)  # (1, 80, T)
    mel_spec = mel_spec.transpose(1, 2)     # (1, T, 80)

model = SimpleSTTModel(
    input_dim=80,
    model_dim=144,
    num_layers=4,
    num_heads=4,
    num_classes=2000  # vocabs에 맞게 지정
)

# 예측
with torch.no_grad():
    logits = model(mel_spec)  # (1, T, num_classes)
    predicted_ids = torch.argmax(logits, dim=-1)  # (1, T)

# 디코딩
id2char = load_vocab("/content/aihub_character_vocabs.csv")
predicted_text = decode_predictions(predicted_ids[0], id2char)

print("예측된 텍스트:", predicted_text)

예측된 텍스트: 델델월쌘렵슐헥슐헥뫼뻑헥헥헥헥헥헥검뫼헥헥미미치럿럿럿럿렵군렵웨딛군3군검군접뻑군3큼검퀸퀸검퀸기헥미헥헥미미럿접기기기열웨웨걷혈혈군혈군군군습미군군군군군군군군군군군군군군군군군군군군군군군군군군겐퀸퀸퀸군군군군군검군빴붉군군검군헥헥검헥읊뻑헥헥얄헥군군검검검휙군쌘귐군사헥퀸퀸헥헥삥멨첩휙촥럿쌘쌘물군쌘쌘차쌘쌘


<ipython-input-67-ef374b5dab2a>:14: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file_path, sr=sr, duration=duration)  # duration을 추가
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
